In [12]:
import zarr
import fsspec
import xarray as xr
from src.utils.mrms.mrms import MRMSAWSS3Client


client = MRMSAWSS3Client()
url = "s3://hrrrzarr/sfc/20200903/20200903_00z_anl.zarr/"
out_dir = "/playpen/mufan/levi/tianlong-chen-lab/nws-lv-precip-forecasting/ccrfcd-gauge-grids/notebooks/hrrr-analysis-test/"
# client.download(url, out_dir, recursive=True)

# Lets grab a forecast file using fsspec
# url = "s3://hrrrzarr/sfc/20200903/20200903_00z_fcst.zarr/2m_above_ground/TMP/2m_above_ground/"
# ds = xr.open_zarr(fsspec.get_mapper(url, anon=True))
# ds.TMP

In [ ]:
import xarray as xr
import s3fs


# Let's grab an analysis file using s3fs
fs = s3fs.S3FileSystem(anon=True)
url = "hrrrzarr/sfc/20200903/20200903_00z_anl.zarr/2m_above_ground/TMP/2m_above_ground/"
url = "hrrrzarr/sfc/20200903/20200903_00z_anl.zarr/2m_above_ground/TMP/2m_above_ground/"

store = s3fs.S3Map(root=url, s3=fs, check=False)
ds = xr.open_zarr(store)
ds.TMP

/home/mufan/miniforge3/envs/torp/lib/python3.10/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


<xarray.DataArray 'TMP' (projection_y_coordinate: 1059,
                         projection_x_coordinate: 1799)> Size: 4MB
dask.array<open_dataset-TMP, shape=(1059, 1799), dtype=float16, chunksize=(150, 150), chunktype=numpy.ndarray>
Dimensions without coordinates: projection_y_coordinate, projection_x_coordinate
Attributes:
    GRIB_PARAM:  [2, 0, 0, 0]
    long_name:   2m_above_ground/TMP
    units:       K

In [ ]:
import pandas as pd

fp = "data/events/2021-01-01_2025-07-25_all_events.csv"
df = pd.read_csv(fp)

,start_time,end_time,station_id,lat,lon,gauge_qpe,mrms_qpe,delta_qpe,cum_gauge_qpe,cum_mrms_qpe
0,2021-01-23 23:00:00,2021-01-24 00:00:00,4709,35.925000,244.883000,0.00,0.003937,-0.003937,0.00,0.003937
1,2021-01-23 23:00:00,2021-01-24 00:00:00,4564,36.028250,244.996361,0.04,0.000000,0.040000,0.04,0.000000
2,2021-01-23 23:00:00,2021-01-24 00:00:00,4779,36.021861,245.041250,0.04,0.000000,0.040000,0.04,0.000000
3,2021-01-23 23:00:00,2021-01-24 00:00:00,4984,35.729250,244.811972,0.00,0.011811,-0.011811,0.00,0.011811
4,2021-01-23 23:00:00,2021-01-24 00:00:00,4724,35.939417,244.922250,0.00,0.015748,-0.015748,0.00,0.015748
...,...,...,...,...,...,...,...,...,...,...
2342106,2025-07-20 20:52:00,2025-07-20 21:52:00,3301,35.995444,245.136194,0.01,0.000000,0.010000,96.48,535.342526
2342107,2025-07-20 20:54:00,2025-07-20 21:54:00,3301,35.995444,245.136194,0.01,0.000000,0.010000,96.49,535.342526
2342108,2025-07-20 20:56:00,2025-07-20 21:56:00,3301,35.995444,245.136194,0.01,0.000000,0.010000,96.50,535.342526
2342109,2025-07-20 20:58:00,2025-07-20 21:58:00,3301,35.995444,245.136194,0.01,0.000000,0.010000,96.51,535.342526


In [13]:
import os
from glob import glob


CLIENT = MRMSAWSS3Client()
WHITELIST = ["3.2"]

url = "s3://hrrrzarr/sfc/20200903/20200903_00z_anl.zarr/"
out_dir = "/playpen/mufan/levi/tianlong-chen-lab/nws-lv-precip-forecasting/ccrfcd-gauge-grids/notebooks/hrrr-analysis-test/"


def download_zarr_and_del_extra_grids(url: str, to: str) -> None:

    # 1. download all paths
    CLIENT.download(url, out_dir, recursive=True)

    # 2. remove extra grid cells
    # paths to grid-cells 
    fps = glob(f"{out_dir}/*/*/*/*/*")
    for fp in fps:
        grid_cell = fp.split("/")[-1]
        if grid_cell not in WHITELIST:
            os.remove(fp)


download_zarr_and_del_extra_grids(url, out_dir)